In [ ]:
#import necessary python libraries
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
mpl.rcParams['figure.figsize'] = (10,8)
mpl.rcParams['axes.grid'] = False

In [ ]:
def load_data(data_path):
  dataset = pd.read_csv(data_path, header = 0, sep = ' ',
            names = ['x','y','dev1/dev2','orient','tx_pwr','col','rssi_1','rssi_2','rssi_3','rssi_4','rssi_5'])
  dataset.set_index(['x','y'])
  return dataset

In [ ]:
data_set = load_data("/content/omni_variable_txpower.txt")

In [ ]:
grouped = data_set.groupby(['x','y'])

In [ ]:
locations = set()
for i in zip(data_set['x'],data_set['y']):
  locations.add(i)

In [ ]:
data_sets = []
for k in list(locations)[:2]:
  data_sets.append(grouped.get_group(k))


In [ ]:
rssi1 = data_sets[0]['rssi_1']
rssi2 = data_sets[0]['rssi_2']
rssi3 = data_sets[0]['rssi_3']
rssi4 = data_sets[0]['rssi_4']
rssi5 = data_sets[0]['rssi_5']
rssi6 = data_sets[1]['rssi_1']
rssi7 = data_sets[1]['rssi_2']
rssi8 = data_sets[1]['rssi_3']
rssi9 = data_sets[1]['rssi_4']
rssi10 = data_sets[1]['rssi_5']
rssi11 = data_sets[1]['rssi_1']

In [ ]:
rssi1.plot(grid=True)
rssi2.plot(grid=True)
rssi3.plot(grid=True)
rssi4.plot(grid=True)
rssi5.plot(grid=True)
rssi6.plot(grid=True)
rssi7.plot(grid=True)
rssi8.plot(grid=True)
rssi9.plot(grid=True)
rssi10.plot(grid=True)
rssi11.plot(grid=True)

In [ ]:
df_input = [data_sets[0][['tx_pwr','rssi_1']],
data_sets[0][['tx_pwr','rssi_2']],
data_sets[0][['tx_pwr','rssi_3']],
data_sets[0][['tx_pwr','rssi_4']],
data_sets[0][['tx_pwr','rssi_5']],
data_sets[1][['tx_pwr','rssi_1']],
data_sets[1][['tx_pwr','rssi_2']],
data_sets[1][['tx_pwr','rssi_3']],
data_sets[1][['tx_pwr','rssi_4']],
data_sets[1][['tx_pwr','rssi_5']],
data_sets[1][['tx_pwr','rssi_1']]]


In [ ]:
#scaling the dataset
def scale_dataset(df_ip):
  scaler = MinMaxScaler()
  return [scaler, scaler.fit_transform(df_ip)]
scaler_set = []
data_scaled = []
for df in df_input:
  scaler, data = scale_dataset(df)
  scaler_set.append(scaler)
  data_scaled.append(data)

In [ ]:
def get_features(scaled_data):
  features = scaled_data
  target = scaled_data[:,1]
  return [features, target]

In [ ]:
feature_target_set = []
for data in data_scaled:
  feature_target_set.append(get_features(data))

In [ ]:
def split_dataset(features, target):
  x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state=123, shuffle=False)
  x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state=123, shuffle=False)
  return [x_train, y_train, x_test, y_test, x_val, y_val]

In [ ]:
train_set = []
test_set = []
validation_set = []
for data in feature_target_set:
  result = split_dataset(data[0], data[1])
  train_set.append([result[0],result[1]])
  test_set.append([result[2],result[3]])
  validation_set.append([result[4],result[5]])

In [ ]:
window_length = 5
batch_size = 32
num_features = 2

In [ ]:
def generator(x,y):
  return TimeseriesGenerator(x,y, length = window_length, sampling_rate=1, batch_size=batch_size)
  

In [ ]:
train_generator = []
test_generator = []
val_generator = []
for train in train_set:
  train_generator.append(generator(train[0],train[1]))
for test in test_set:
  test_generator.append(generator(test[0],test[1]))
for val in validation_set:
  val_generator.append(generator(val[0],val[1]))

In [ ]:

model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128,input_shape=(window_length, num_features),return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.LSTM(64, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.summary()

In [ ]:
def train_model(train_gen, val_gen):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
  model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics=[tf.metrics.MeanAbsoluteError()])
  history = model.fit_generator(train_gen, epochs=100,
                              validation_data=val_gen,
                              shuffle=False,
                              callbacks=[early_stopping])
  return [model,history]

In [ ]:
def train_val_plot(history):
  plt.plot(history.history['loss'],label='Training Loss')
  plt.plot(history.history['val_loss'],label='Validation loss')
  plt.title(f"model_{i+1} Tx = {10 + i}")
  plt.legend()
  plt.show()

In [ ]:
def evaluate(test_gen, model):
  return model.evaluate_generator(test_gen, verbose = 0)

In [ ]:
def make_prediction(model, test_gen):
  return model.predict_generator(test_gen)

In [ ]:
def actual_pred(predictions, x_test, df_input, scaler, i):
  df_pred = pd.concat([pd.DataFrame(x_test[:,:-1][window_length:]),pd.DataFrame(predictions)],axis=1)
  reverse_trans = scaler.inverse_transform(df_pred)
  df_final = df_input[-predictions.shape[0]:]
  df_final['pred'] = reverse_trans[:,-1]
  df_final[[f'rssi_{(i % 5) + 1}','pred']].plot(figsize=(10,8))
  plt.title(f"model_{i+1} Tx = {10+i}")
  plt.legend()
  plt.show()

In [ ]:
import pickle

models = []
histories = []
evaluations = []
predictions  = []
for i in range(11):
  model, history = train_model(train_generator[i],val_generator[i])
  models.append(model)
  histories.append(history)
  evaluations.append(evaluate(test_generator[i],model))
  prediction = make_prediction(model, test_generator[i])
  predictions.append(prediction)
  file_name = f'saved_model_{10+i}.pkl'
  pickle.dump(model, open(file_name, 'wb'))



In [ ]:
for i in range(11):
  train_val_plot(histories[i])

In [ ]:
for i in range(11):
  actual_pred(predictions[i], test_set[i][0], df_input[i], scaler_set[i], i)  